In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import os
import cv2
import time

In [2]:
data_dir = r'gaf_images'
categories = ['long', 'short', 'neutral']

In [3]:
tm = time.time()

data = []
labels = []

for category in categories:
    ctr = 0
    path = os.path.join(data_dir, category)
    class_num = categories.index(category)
    for img in os.listdir(path):
        ctr+=1
        if ctr == 1200:
            break
        
        img_path = os.path.join(path, img)
        img_array = cv2.imread(img_path)
        data.append(img_array)
        labels.append(class_num)
    print(ctr)
print(time.time()-tm)

436
368
365
0.375537633895874


In [4]:
data = np.array(data) / 255.0
labels = np.array(labels)
print(labels.shape, data.shape)

(1169,) (1169, 60, 60, 3)


In [5]:
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

In [6]:
X_train = np.array(X_train)
X_val = np.array(X_val)
y_train = np.array(y_train)
y_val = np.array(y_val)

In [7]:
# train_datagen = ImageDataGenerator()
# val_datagen = ImageDataGenerator()

# train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
# val_generator = val_datagen.flow(X_val, y_val, batch_size=32)

In [8]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(60, 60, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


C:\Users\prast\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 58, 58, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 29, 29, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 27, 27, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 13, 13, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 11, 11, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 5, 5, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 3200)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         409,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 503,363 (1.92 MB)

 Trainable params: 503,363 (1.92 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
from tensorflow.keras.utils import to_categorical
y_lab_oh = to_categorical(y_train, 3)

In [11]:
y_val_oh = to_categorical(y_val, 3)

In [12]:
print(X_train.shape)

(935, 60, 60, 3)


In [13]:
model

<Sequential name=sequential, built=True>

In [16]:
batch_size = 32  # Your batch size

# Calculate steps per epoch and validation steps
# steps_per_epoch = len(train_generator)
# validation_steps = len(val_generator)

# Fit the model
model.fit(X_train, y_lab_oh,validation_data=(X_val, y_val_oh),
    epochs=20
)
# print(steps_per_epoch)
# print(validation_steps)

Epoch 1/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 94ms/step - accuracy: 0.5281 - loss: 0.9512 - val_accuracy: 0.3547 - val_loss: 1.2577
Epoch 2/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 4s 120ms/step - accuracy: 0.5298 - loss: 0.9589 - val_accuracy: 0.3547 - val_loss: 1.2587
Epoch 3/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 4s 142ms/step - accuracy: 0.6158 - loss: 0.8142 - val_accuracy: 0.3333 - val_loss: 1.3122
Epoch 4/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - accuracy: 0.6750 - loss: 0.7337 - val_accuracy: 0.3291 - val_loss: 1.4970
Epoch 5/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 4s 116ms/step - accuracy: 0.7047 - loss: 0.6695 - val_accuracy: 0.3632 - val_loss: 1.3898
Epoch 6/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 4s 138ms/step - accuracy: 0.8070 - loss: 0.5506 - val_accuracy: 0.3248 - val_loss: 1.7426
Epoch 7/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 4s 77ms/step - accuracy: 0.8333 - loss: 0.4024 - val_accuracy: 0.3718 - val_loss: 1.9715
Epoch 8/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 4s 145ms/step - accuracy: 0.8911 - loss: 0.3188 - val_accuracy: 0.346